In [3]:
!pip install ultralytics
!pip install torch torchvision
!pip install opencv-python
!pip install zipfile36

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.8/874.8 kB 23.1 MB/s eta 0:00:00


In [6]:
import torch
import os
import zipfile
from ultralytics import YOLO
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from pathlib import Path

# Step 1: Dataset Handling
class DomainDataset(torch.utils.data.Dataset):
    def __init__(self, image_folder, label_folder=None, transform=None):
        self.image_folder = Path(image_folder)
        self.label_folder = Path(label_folder) if label_folder else None
        self.image_paths = list(self.image_folder.glob("**/*.jpg"))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')

        label_path = self.get_label_path(img_path) if self.label_folder else None
        label = self.load_label(label_path) if label_path else None

        if self.transform:
            img = self.transform(img)

        return img, label

    def get_label_path(self, img_path):
        label_name = img_path.stem + ".txt"
        return self.label_folder / label_name

    def load_label(self, label_path):
        with open(label_path, 'r') as f:
            label = f.readlines()
        return label  # Modify this to return appropriate label data

# Step 2: Model Setup for Domain Adaptation
class YOLOv8DomainAdaptation:
    def __init__(self, source_weights, synthetic_dataset, real_dataset, test_dataset, lr=1e-4, epochs=10, batch_size=16):
        self.model = YOLO(source_weights)
        self.synthetic_dataset = synthetic_dataset
        self.real_dataset = real_dataset
        self.test_dataset = test_dataset
        self.epochs = epochs
        self.lr = lr
        self.batch_size = batch_size
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

    def pretrain_on_synthetic(self):
        print("Pretraining on synthetic data...")
        synthetic_loader = DataLoader(self.synthetic_dataset, batch_size=self.batch_size, shuffle=True)
        self.train_model(synthetic_loader, phase="Synthetic Pretraining")

    def fine_tune_on_real(self):
        print("Fine-tuning on real data...")
        real_loader = DataLoader(self.real_dataset, batch_size=self.batch_size, shuffle=True)
        self.train_model(real_loader, phase="Fine-tuning on Real Data")

    def train_model(self, data_loader, phase="Training"):
        self.model.train()
        for epoch in range(self.epochs):
            total_loss = 0
            for imgs, labels in tqdm(data_loader, desc=f'{phase} Epoch {epoch+1}/{self.epochs}'):
                imgs, labels = imgs.to(self.model.device), labels  # Assuming labels are not tensor (adjust accordingly)

                # Forward pass through YOLO model
                results = self.model(imgs)

                # Here, 'results' includes predictions; modify the following lines to compute the loss
                loss = self.compute_loss(results, labels)
                total_loss += loss.item()

                # Backpropagation
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            print(f"{phase} Epoch [{epoch+1}/{self.epochs}], Loss: {total_loss/len(data_loader)}")

    def compute_loss(self, predictions, labels):
        # YOLOv8 handles the loss internally during training
        # You can extend this if needed, but YOLOv8 by default computes its own object detection loss
        return torch.tensor(0)  # Modify this to calculate the actual loss if necessary

    def evaluate_model(self):
        print("Evaluating on test data...")
        test_loader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)
        self.model.eval()
        with torch.no_grad():
            for imgs, labels in tqdm(test_loader, desc="Evaluating"):
                imgs = imgs.to(self.model.device)
                results = self.model(imgs)
                # Modify the following to compute and store evaluation metrics (e.g., mAP, precision, etc.)
                self.process_results(results, labels)

    def process_results(self, results, labels):
        # Add logic to compare results with ground truth labels and compute evaluation metrics
        pass

    def save_model(self, path):
        self.model.save(path)

# Step 3: Utility Functions
# def extract_zip(file_path, extract_to):
#     print(f"Extracting {file_path}...")
#     with zipfile.ZipFile(file_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_to)

# Step 4: Main Function
if __name__ == "__main__":
    # Paths to your datasets (adjust these paths)
    synthetic_zip_path = 'path_to_synthetic.zip'
    train_zip_path = 'path_to_train.zip'
    test_zip_path = 'path_to_test.zip'
    extract_dir = 'extracted_data'

    # Extract datasets
    extract_zip(synthetic_zip_path, extract_dir)
    extract_zip(train_zip_path, extract_dir)
    extract_zip(test_zip_path, extract_dir)

    # Define image and label paths for each dataset
    synthetic_image_folder = os.path.join(extract_dir, 'synthetic/images')
    synthetic_label_folder = os.path.join(extract_dir, 'synthetic/labels')

    real_image_folder = os.path.join(extract_dir, 'train/images')
    real_label_folder = os.path.join(extract_dir, 'train/labels')

    test_image_folder = os.path.join(extract_dir, 'test/images')
    test_label_folder = os.path.join(extract_dir, 'test/labels')

    # Transform for the datasets
    transform = transforms.Compose([
        transforms.Resize((640, 640)),
        transforms.ToTensor(),
    ])

    # Create datasets
    synthetic_dataset = DomainDataset(synthetic_image_folder, synthetic_label_folder, transform=transform)
    real_dataset = DomainDataset(real_image_folder, real_label_folder, transform=transform)
    test_dataset = DomainDataset(test_image_folder, test_label_folder, transform=transform)

    # Initialize YOLOv8 domain adaptation model
    yolo_adaptation = YOLOv8DomainAdaptation(
        source_weights='yolov8.pt',
        synthetic_dataset=synthetic_dataset,
        real_dataset=real_dataset,
        test_dataset=test_dataset,
        lr=1e-4,
        epochs=10,
        batch_size=16
    )

    # Train the model
    yolo_adaptation.pretrain_on_synthetic()
    yolo_adaptation.fine_tune_on_real()

    # Evaluate the model
    yolo_adaptation.evaluate_model()

    # Save the model
    yolo_adaptation.save_model('yolov8_domain_adapted.pt')

Extracting path_to_synthetic.zip...


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_synthetic.zip'